#### Import Julia packages

In [47]:
using Gridap
using GridapMakie, CairoMakie, FileIO
using Gridap.FESpaces
using Gridap.ReferenceFEs
using Gridap.Arrays
using Gridap.Algebra
using Gridap.Geometry
using Gridap.Fields
using Gridap.CellData
using FillArrays
using Test
using InteractiveUtils
using PyPlot

In [48]:
# using Pkg
# Pkg.add("PyPlot")


function mark_nodes(f,model::DiscreteModel)
  topo   = get_grid_topology(model)
  coords = get_vertex_coordinates(topo)
  mask = map(f,coords)
  return mask
end
function update_labels!(e::Integer,model::CartesianDiscreteModel,f_Γ::Function,name::String)
    mask = mark_nodes(f_Γ,model)
    _update_labels_locally!(e,model,mask,name)
    nothing
end
function _update_labels_locally!(e,model::CartesianDiscreteModel{2},mask,name)
  topo   = get_grid_topology(model)
  labels = get_face_labeling(model)
  cell_to_entity = labels.d_to_dface_to_entity[end]
  entity = maximum(cell_to_entity) + e
  # Vertices
  vtxs_Γ = findall(mask)
  vtx_edge_connectivity = Array(get_faces(topo,0,1)[vtxs_Γ])
  # Edges
  edge_entries = [findall(x->any(x .∈  vtx_edge_connectivity[1:end.!=j]),
    vtx_edge_connectivity[j]) for j = 1:length(vtx_edge_connectivity)]
  edge_Γ = unique(reduce(vcat,getindex.(vtx_edge_connectivity,edge_entries),init=[]))
  labels.d_to_dface_to_entity[1][vtxs_Γ] .= entity
  labels.d_to_dface_to_entity[2][edge_Γ] .= entity
  add_tag!(labels,name,[entity])
  return cell_to_entity
end

_update_labels_locally! (generic function with 1 method)

#### Import Custom functions

In [49]:
#include("Julia_functions/solution_animation.jl")
include("Julia_functions/indicator_chi.jl")
include("Julia_functions/cost_fun.jl")
include("Julia_functions/gradient_descent.jl")
include("Julia_functions/solvers.jl")
include("Julia_functions/find.jl")
include("Julia_functions/res_plot.jl")

result_plot (generic function with 1 method)

#### Boundary conditions

In [50]:
# Funksjon for å definere vinduer: x[1] i intervallet [-0.25, 0.25] og x[2] på y = -1 (nedre kant)
f_Γ_w(x) = (-0.25 ≤ x[1] ≤ 0.25) && (x[2] ≈ -1)

# Funksjon for å definere dører: x[2] i intervallet [-0.10, 0.10] og x[1] på x = -1 (venstre kant)
f_Γ_d(x) = (-0.10 ≤ x[2] ≤ 0.10) && (x[1] ≈ -1)

# Definer vegger som de områdene som ikke er vinduer eller dører
f_Γ_wall(x) = !(f_Γ_w(x) || f_Γ_d(x))



f_Γ_wall (generic function with 1 method)

#### Domain & Discretization

In [51]:
domain = (-1,+1,-1,+1)
partition = (20,20)
model = CartesianDiscreteModel(domain,partition)
update_labels!(1, model, f_Γ_w, "window")
update_labels!(2, model, f_Γ_d, "door")
update_labels!(3, model, f_Γ_wall, "wall")

order = 1
degree = 2*order

Ω = Triangulation(model)
dΩ = Measure(Ω, degree)

Γ_w = BoundaryTriangulation(model, tags="window")
dΓ_w = Measure(Γ_w, degree)

Γ_d = BoundaryTriangulation(model, tags="door")
dΓ_d = Measure(Γ_d, degree)

Γ_wall = BoundaryTriangulation(model, tags="wall")
dΓ_wall = Measure(Γ_wall, degree)

reffe = ReferenceFE(lagrangian,Float64,order)
Testspace = TestFESpace(model,reffe,conformity=:H1)                         
Trialspace = TransientTrialFESpace(Testspace)                                
Uspace = FESpace(model, reffe, conformity=:H1)                                           

UnconstrainedFESpace()

#### Time parameters

In [52]:
t0 = 0.0    # Start time
tF = 10.0   # End time
Δt = 0.05   # Timestep

0.05

#### Room parameters

In [53]:

Toutdoor(x,t)=5.0
Tout(t)=x->Toutdoor(x,t)

price(t)=1.0

Tini(x)=20.0

TIni=interpolate_everywhere(Tini, Uspace(t0))
Tfin=interpolate_everywhere(20.0, Uspace(tF))

SingleFieldFEFunction():
 num_cells: 400
 DomainStyle: ReferenceDomain()
 Triangulation: BodyFittedTriangulation()
 Triangulation id: 16041350878974327689

#### Controll parameter

In [54]:
q_pos(x) = χ(x[1], -0.9, -0.7) * χ(x[2], 0, 0.20) + χ(x[1], 0.2, 0.4) * χ(x[2], -0.5, 0)
Q(x,t)=χ(t,0.0,3.0)*100.0*q_pos(x)
Qt(t)=x->Q(x,t)

Qt (generic function with 1 method)

#### Solver parameters

In [55]:
L2norm(u)=√((tF-t0)*∑(Δt*∑(∫(uu⋅uu)*dΩ) for (t,uu) in u))
L2skp(u)=(tF-t0)*∑(Δt*∑(∫(uu⋅uu)*dΩ) for (t,uu) in u)
ls = LUSolver()
θ = 0.5
solver = ThetaMethod(ls, Δt, θ)
γ = 1

Proj(a,b,z) = min(max(a,z),b)
# Proj(z) = map(x->Proj(a,b,x),z)
a=0.0
b=1000.0
Proj(z) = [(t,FEFunction(Uspace,map(x->Proj(a,b,x), get_free_dof_values(zz)))) for (t,zz) in z]
# [(t,interpolate_everywhere(map(x->P(a,b,x),get_free_dof_values(zz)),Uspace(t))) for (t,zz) in z]


Proj (generic function with 2 methods)

#### SE Test

In [56]:
h_wall(x) = 1.0
h_window(x) = 1.0
h_door(x) = 1.0

ρ(x)=1.225
c(x)=1.0#1020.0
k(x)=1.0#15.0

h = h_wall, h_window, h_door
constants = (c, ρ, k, h) 


dΓ = dΓ_w, dΓ_d, dΓ_wall

ls = LUSolver()
θ = 0.5
solver = ThetaMethod(ls, Δt, θ)

solution, _, _ = SEsolver(solver, Qt, Trialspace, Testspace; dΩ, dΓ, Tout, constants) 


ErrorException: invalid redefinition of constant Main.h

In [57]:
# Opprett en mappe for midlertidige filer hvis den ikke eksisterer
if !isdir("tmp")
    mkdir("tmp")
end

# Initialtilstanden og resten av tidsløsningene
uh0 = solution[1][2]  # Første element er (t0, T0), vi henter T0
uh  = solution[2:end]  # De resterende løsningene

# Lagre resultater i Paraview-format (VTK)
createpvd("results") do pvd
    # Første løsning (t=0)
    pvd[0] = createvtk(Ω, "tmp/results_0.vtu", cellfields=["u" => uh0])

    # Lagrer løsninger for hver tidssteg
    for (tn, uhn) in uh
        pvd[tn] = createvtk(Ω, "tmp/results_$tn.vtu", cellfields=["u" => uhn])
    end
end


UndefVarError: UndefVarError: `solution` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

#### Optimize

In [58]:
include("Julia_functions/solution_animation.jl")
include("Julia_functions/indicator_chi.jl")
include("Julia_functions/cost_fun.jl")
include("Julia_functions/gradient_descent.jl")
include("Julia_functions/solvers.jl")
include("Julia_functions/find.jl")

(ys,qs,ps,costs) = GradientDescent(;solveSE=SEsolver, 
solveAE=AEsolver, 
spaces=(Trialspace, Testspace, Uspace), 
dΩ=dΩ, 
dΓ=dΓ, 
Q=Qt, 
J=E, 
∇f=∇e, 
P=Proj, 
s_min=s_min,
sminargs=nothing, 
saveall=true, 
tol=1e-5, 
iter_max=2, 
Δt=Δt, 
t0=t0, 
tF=tF)

UndefVarError: UndefVarError: `dΓ` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

#### Results

In [59]:
#TODO: lag plot; pris, pådrag, temperatur 

In [60]:
result_plot()